In [13]:
from neuron import h, gui
import bokeh as bk
import bokeh.plotting as bkp
import matplotlib.pyplot as plt
import numpy as np
from bokeh.palettes import Dark2_5 as palette
import itertools 
import ipywidgets as widgets

colors = itertools.cycle(palette)  
%matplotlib inline
bkp.output_notebook()

Loading BokehJS ...

In [2]:
# define branched cable class
class branched_cable(object):
    def __init__(self,):
        '''initialize dictionaries for storing information about cell'''
        self.segments = {}
        self.stims = {}
        self.record_vecs = {}
           
    def make_cable(self,L,diam,segname='cable',cm = 1, Ra = 150,Rm = 2800, Vrest = -65.,nseg=None):
        '''make a cable with specified distributed properties (approximate a sphere by setting diam=L= x)'''
        cable = h.Section(name=segname,cell=self)
        cable.L = L #micro-m
        cable.diam = diam # micro-m
        cable.cm = cm  # picofarads/cm^2
        cable.Ra = Ra # megaOhms/cm^2
        
        if nseg is None: # use d_lambda rule to get good guess of best number of segments
            cable.nseg = 10*(int((L/(0.1*h.lambda_f(100))+.999)/2.)*2 + 1)
        else:
            cable.nseg = nseg

        cable.insert('pas')
        cable.g_pas = 1/Rm # picoSiemens/cm^2
        cable.e_pas = Vrest # milliVolts
        self.segments[segname] = cable
        
        
        
    def add_branch(self,seg,pos,L,diam,segname='branch',cm = 1, Ra = 150,Rm = 2800, Vrest = -65.):
        '''add a branch to seg at pos with specified distributed properites'''
        self.make_cable(L,diam,segname=segname,cm = cm, Ra = Ra,Rm = Rm, Vrest = Vrest)
        self.segments[segname].connect(self.segments[seg](pos))
        #self.segments[segname]
                  
    def add_IClamp(self,seg,pos,amp,dur,delay = 50,stimname = 'iclamp'):
        '''insert current clamp on cell compartment seg at position pos (0 to 1: percent down the process) 
        amp: amplitude (milliamps)
        dur: duration (milliseconds)
        delay: delay before stimulus (milliseconds)'''
        iclamp = h.IClamp(self.segments[seg](pos)) 
        iclamp.amp = amp
        iclamp.dur= dur
        iclamp.delay = delay
        self.stims[stimname]=iclamp
        
    def add_stim(self,seg,pos):
        '''add a synapse on cell compartment seg at position pos (0 to 1: percent down the process)
        
        Not fully functional yet'''
        stim = h.NetStim() # Make a new stimulator

        # Attach it to a synapse in the middle of the dendrite
        # of the first cell in the network. (Named 'syn_' to avoid
        # being overwritten with the 'syn' var assigned later.)
        syn_ = h.ExpSyn(self.segments[seg](pos), name='syn_')

        stim.number = 1
        stim.start = 9
        ncstim = h.NetCon(stim, syn_)
        ncstim.delay = 1
        ncstim.weight[0] = 0.04 # NetCon weight is a vector.
        
    def recording_vecs(self,locations):
        ''' add NEURON vectors to record desired parameters specified by list of lists locations
        
        each entry in locations is a list with 3 entries: segment name, location on segment (0-1), and the variable 
        name you wish to record (e.g. v for membrane potential)'''
        # locations is list of lists with segname, location, variable
        currKeys = self.record_vecs.keys()
        if 'time' not in currKeys:
            t_vec = h.Vector()
            t_vec.record(h._ref_t)
            self.record_vecs['time']= t_vec
        
        for v in locations:
            try:
                keyname = "%s:%.2f:%s" % (v[0],v[1],v[2])
            except:
                raise("invalid specification of recording locations")
                
            
            if keyname in currKeys:
                print(keyname+"is a already a recording vec")
                
            else:
                self.record_vecs[keyname] = h.Vector()
                eval("self.record_vecs[keyname].record(self.segments[v[0]](v[1])._ref_" + v[2]+")")   
                
def run_current_clamp(cell,stimname,stimVals,tstop = 130):
    
    results = {}
    for i, amp in enumerate(stimVals):
        cell.stims[stimname].amp = amp
        h.tstop=tstop
        h.run()
        
        if i == 0:
            t = cell.record_vecs['time'].as_numpy()
            for key in cell.record_vecs.keys():
                results[key] = np.zeros([t.shape[0],len(stimVals)])
        
        for key in cell.record_vecs.keys():
            results[key][:,i] = cell.record_vecs[key].as_numpy()
            
    return t, results

In [44]:
def plot_single_compartment_IClamp(diam=10,Rm=2800,cm=1):
    
    #t, vArr, I = run_single_compartment_IClamp(diam,Rm,cm)
    cell = branched_cable()
    cell.make_cable(diam,diam,nseg = 1,Rm=Rm,cm=cm)
    cell.add_IClamp('cable',.5,.001,25,delay = 20)
    cell.recording_vecs([['cable',.5,'v']])

    I = np.linspace(-.2,.2,num=7)
    t, results = run_current_clamp(cell,'iclamp',I,tstop=80)
    
    vArr =  results['cable:0.50:v']
    
    RC= bkp.figure(plot_width=400,plot_height=400)
    RC.xaxis.axis_label="time"
    RC.yaxis.axis_label="V (mV)"
    colors = itertools.cycle(palette)  
    
    
    for i, color in zip(range(vArr.shape[1]),colors):
        RC.line(t,vArr[:,i],color=color) #,legend=str(I[i]))
    #RC.legend.location="top_right"
    
    #find steady state voltage
    VI = bkp.figure(plot_height=400,plot_width=400)
    VI.xaxis.axis_label="I (mA)"
    VI.yaxis.axis_label="V (mV)"
    ss_ind = np.where(t>43)[0][0]
    V_ss = vArr[ss_ind,:]
    VI.line(I,V_ss)
    
    
    # time constant
    vArr = vArr +65
    V_half = (V_ss+65.)/2.
    tau = np.zeros([7,])
    for col in range(I.size):
        tau[col] = np.where(vArr[:,i]>V_half[i])[0][0]/t.size*80. -20
        
    T = bkp.figure(plot_height=400,plot_width=400)
    T.xaxis.axis_label="I (mA)"
    T.yaxis.axis_label= "tau V_1/2"
    T.line(I,tau)
    
    bkp.show(bk.layouts.layout([[RC,VI],[T]]))
    
             




In [43]:
widgets.interact_manual(plot_single_compartment_IClamp,
                 diam=widgets.FloatSlider(min=.1,max=100,continuous_update=False),
                 Rm=widgets.FloatSlider(min=100,max=5000,continuous_update=False),
                 cm = widgets.FloatSlider(min=.1,max=10,continous_update=False))

A Jupyter Widget

<function __main__.plot_single_compartment_IClamp>

# Steady State Properties
## V-I curve analysis

In [9]:
I = np.linspace(-.2,.2,num=7)
def get_VI(diam,Rm,cm):
    cell = branched_cable()
    cell.make_cable(diam,diam,nseg = 1,Rm=Rm,cm=cm)
    cell.add_IClamp('cable',.5,.001,25,delay = 20)
    cell.recording_vecs([['cable',.5,'v']])

    t, results = run_current_clamp(cell,'iclamp',I,tstop=80)
    
    vArr =  results['cable:0.50:v']
    ss_ind = np.where(t>43)[0][0]
    V_ss = vArr[ss_ind,:]
    
    slope = np.diff(V_ss).mean()
    return V_ss,slope

In [21]:
# V-I slope vs diam
diams = np.linspace(10,100,num=50)
V_ss = np.zeros([I.shape[0],50])
slope = np.zeros([50,])
for i, diam in enumerate(list(diams)):  
    V_ss[:,i], slope[i] = get_VI(diam,2800,1)
    
VI = bkp.figure(plot_height=400,plot_width=400)
for i, color in zip(range(50),colors):
    VI.line(I,V_ss[:,i],color=color)
VI.xaxis.axis_label="I (mA)"
VI.yaxis.axis_label="mV"
    
S = bkp.figure(plot_height=400,plot_width=400)
S.yaxis.axis_label="V-I slope"
S.xaxis.axis_label="diameter"
S.line(diams,slope)
bkp.show(bk.layouts.layout([[VI,S]]))



In [23]:
# V-I slope vs Rm
Rms = np.logspace(2,5,num=50)
V_ss = np.zeros([I.shape[0],50])
slope = np.zeros([50,])
for i, Rm in enumerate(list(Rms)):  
    V_ss[:,i], slope[i] = get_VI(20,Rm,1)
    
VI = bkp.figure(plot_height=400,plot_width=400)
for i, color in zip(range(50),colors):
    VI.line(I,V_ss[:,i],color=color)
VI.xaxis.axis_label="I (mA)"
VI.yaxis.axis_label="mV"
    
S = bkp.figure(plot_height=400,plot_width=400)
S.yaxis.axis_label="V-I slope"
S.xaxis.axis_label="Rm (MOhm)"
S.line(Rms,slope)
bkp.show(bk.layouts.layout([[VI,S]]))

# V-i slope vs Cm

In [29]:
# V-i slope vs Cm
cms = np.linspace(.01,5,num=50)
V_ss = np.zeros([I.shape[0],50])
slope = np.zeros([50,])
for i, cm in enumerate(list(cms)):  
    V_ss[:,i], slope[i] = get_VI(20,2800,cm)
    
VI = bkp.figure(plot_height=400,plot_width=400)
for i, color in zip(range(50),colors):
    VI.line(I,V_ss[:,i],color=color)
VI.xaxis.axis_label="I (mA)"
VI.yaxis.axis_label="mV"
    
S = bkp.figure(plot_height=400,plot_width=400)
S.yaxis.axis_label="V-I slope"
S.xaxis.axis_label="Cm (pF)"
S.line(cms,slope)
bkp.show(bk.layouts.layout([[VI,S]]))



# Dynamics
## V_1/2 analysis


In [ ]:
I = .2
def get_Vhalf(diam,Rm,cm):
    cell = branched_cable()
    cell.make_cable(diam,diam,nseg = 1,Rm=Rm,cm=cm)
    cell.add_IClamp('cable',.5,.001,25,delay = 20)
    cell.recording_vecs([['cable',.5,'v']])

    t, results = run_current_clamp(cell,'iclamp',I,tstop=80)
    vArr =  results['cable:0.50:v']
    
    vZero = vArr+65
    ss_ind = np.where(t>43)[0][0]
    V_ss = vZero[ss_ind]
    V_half = .5*V_ss
    tau = np.where(vZero>V_half)[0][0]/t.size*80. -20
    
    return vArr, tau

In [ ]:
# Vary Rm

# raw plot

# tau v Rm

In [ ]:
# Vary Cm

# raw plot

# tau v Cm

In [20]:
# make a cable, insert stimulus somewhere

cell = branched_cable()
cell.make_cable(120,10)
cell.add_IClamp('cable',.9,.001,50)

In [21]:
# make branched cable, insert stimulus on branch

cell = branched_cable()
cell.make_cable(120,10)
cell.add_branch('cable',.5,50,5)
cell.add_IClamp('branch',.5,.001,50)
cell.recording_vecs([['cable',.1,'v'],['cable',.5,'v'],['branch',.5,'v']])
# record along entire length of cable

In [22]:
# run simulation
h.tstop = 500
h.run()

0.0

In [23]:
f= bkp.figure()
colors = itertools.cycle(palette)  

keys = list(cell.record_vecs.keys())
print(keys)
keys.remove('time')
for key, color in zip(keys,colors):
    print(key)
    f.line(cell.record_vecs['time'].as_numpy(),cell.record_vecs[key].as_numpy(),color=color,legend=key)
    f.legend.location="top_right"
    
bkp.show(f)

['time', 'cable:0.10:v', 'cable:0.50:v', 'branch:0.50:v']
cable:0.10:v
cable:0.50:v
branch:0.50:v


	-65 


In [ ]:
# electrode in middle of cable

# V vs pos, fit exponential for length constant

# RC response for various positions

# V-I curve for different positions

# repeat with different diameters

# repeat changing membrane resistance

In [ ]:
# repeat above with branch point

# recording point before and after branch

# change size of branch point relative to main cable

In [12]:
# spherical cell: insert mechanisms to a cell one at a time

# passive
cell = branched_cable()
cell.make_cable(12,12)
cell.add_IClamp('cable',.5,1,50)
cell.recording_vecs(['cable',.5,'v'])

# run simulation and plot

In [10]:

# K delayed rectifier
# can currently vary conductance
#     need to add capability to vary kinetics, voltage dependence (half activation and steepness)
cell = branched_cable()
cell.make_cable(12,12)
cell.segments['cable'].g_pas = 0
cell.segments['cable'].insert('kdr')
cell.segments['cable'].gkdrbar_kdr = .003

cell.add_IClamp('cable',.5,1,50)


# run simulation and plot

In [11]:
# K inactivating
# I don't have  a good mod file for this at the moment
#cell = branched_cable()
#cell.make_cable(12,12)
#cell.segments['cable'].g_pas = 0

#cell.add_IClamp('cable',.5,1,50)



In [14]:
# Ih 
# can change conductance and voltage dependence at the moment
cell.segments['cable'].insert('hd')
cell.segments['cable'].ghdbar_hd = .0006
cell.segments['cable'].vhalfl_hd=-82

# run and plot

In [ ]:
# Persistent sodium or rapid activating calcium
# need mod file


In [16]:
# inactivating sodium current
cell.segments['cable'].insert('na3')

cell.segments['cable'].gbar_na3 = .032


In [ ]:
# ball and stick cell
cell = branched_cable()
cell.make_cable(12,12)
cell.add_branch('cable',1,200,1)
cell.add_stim('branch',1)

# add passive channels to dendrites

# active channels to soma

In [ ]:
# add checkboxes for inserting different distributed mechanisms

# plot response 